In [1]:
using JuMP
using Gurobi
using Combinatorics
using ToeplitzMatrices
using LinearAlgebra

In [2]:
# Define parameters of the problem
A=[0.2 0; 0 1.3]
C=[0.5 1.5; 0 1]

n_y,n_x=size(C)

H_w=[1 0; -1 0; 0 1; 0 -1]
h_w=1.5*[1; 1; 1; 1]
n_w=size(h_w,1)


H_v=[1 0; -1 0; 0 1; 0 -1]
h_v=0.3*[1; 1; 1; 1]
n_v=size(h_v,1)

H_x0=[1 0; -1 0; 0 1; 0 -1]
h_x0=1.7*[1; 1; 1; 1]
n_x0=size(h_x0,1)

T=10
N=3

bigM=1e5

100000.0

In [3]:
time_instants=0:T-1 # MODIFIED
all_schedules=collect(combinations(time_instants,N)); # do we need to collect ?
n_schedules=size(all_schedules,1)

120

In [4]:
# compute the J matrix
J=I # Identity matrix
for i = 1:T
    J = [J; (A^i)] # not efficient
end

In [5]:
# compute the S matrix
S=zeros(n_x*(T+1),n_x*T)
l=one(A) # Is it efficient, command "I" is another option...
S[n_x+1:2*n_x,1:n_x]=l # A^0
for i=1:T-1 # loop over the lines
    l=[A*l[1:n_x,1:n_x] l] # A*l[1:nx,1:nx] is A^i
    S[(i+1)*n_x+1:(i+2)*n_x, 1:(i+1)*n_x]=l
end
print("We don't use the triangular structure...\n")

We don't use the triangular structure...


In [6]:
# function to compute C_M
print("We don't use the sparse structure...\n")
function computeC_M(T,M)
    # construct C_sched (=C_bar)
    M=M.+1 # to have indices
    diag=zeros(T)
    diag[M].=1
    dMat=Diagonal(diag)
    C_sched=kron(dMat,C)
    return C_sched
end

We don't use the sparse structure...


computeC_M (generic function with 1 method)

In [7]:
# compute C_bar
print("We don't use the sparse structure...\n")

# preallocation
C_bar=zeros(n_schedules, T*n_y, (T+1)*n_x)

for schedule_idx=1:n_schedules
    M_i=all_schedules[schedule_idx]
    C_bar[schedule_idx,:,1:T*n_x].=computeC_M(T,M_i)
end

We don't use the sparse structure...


In [8]:
# compute H_eta and h_eta
H_eta=[kron(Matrix(1I,T,T), H_w) zeros(T*n_w,T*n_y + n_x) ;
    zeros(T*n_v, T*n_x) kron(Matrix(1I,T,T), H_v) zeros(T*n_v,n_x);
    zeros(n_x0,T*(n_x+n_y)) H_x0]

h_eta=[kron(ones(T),h_w); kron(ones(T),h_v); h_x0];

In [9]:
model = Model(Gurobi.Optimizer)

Academic license - for non-commercial use only - expires 2020-12-31


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi

In [10]:
empty!(model)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Gurobi

In [11]:
# Declare variables
@variable(model,Q[1:n_schedules,1:n_x*T,1:n_x*T])
@variable(model,r[1:n_schedules,1:n_x*T])
@variable(model,Lambda[1:n_schedules,1:2*n_x*(T+1),1:T*(n_w+n_v)+n_x0])
@variable(model,alpha0[1:n_schedules,1:T+1])

120×11 Array{VariableRef,2}:
 alpha0[1,1]    alpha0[1,2]    …  alpha0[1,10]    alpha0[1,11]
 alpha0[2,1]    alpha0[2,2]       alpha0[2,10]    alpha0[2,11]
 alpha0[3,1]    alpha0[3,2]       alpha0[3,10]    alpha0[3,11]
 alpha0[4,1]    alpha0[4,2]       alpha0[4,10]    alpha0[4,11]
 alpha0[5,1]    alpha0[5,2]       alpha0[5,10]    alpha0[5,11]
 alpha0[6,1]    alpha0[6,2]    …  alpha0[6,10]    alpha0[6,11]
 alpha0[7,1]    alpha0[7,2]       alpha0[7,10]    alpha0[7,11]
 alpha0[8,1]    alpha0[8,2]       alpha0[8,10]    alpha0[8,11]
 alpha0[9,1]    alpha0[9,2]       alpha0[9,10]    alpha0[9,11]
 alpha0[10,1]   alpha0[10,2]      alpha0[10,10]   alpha0[10,11]
 alpha0[11,1]   alpha0[11,2]   …  alpha0[11,10]   alpha0[11,11]
 alpha0[12,1]   alpha0[12,2]      alpha0[12,10]   alpha0[12,11]
 alpha0[13,1]   alpha0[13,2]      alpha0[13,10]   alpha0[13,11]
 ⋮                             ⋱                  ⋮
 alpha0[109,1]  alpha0[109,2]     alpha0[109,10]  alpha0[109,11]
 alpha0[110,1]  alpha0[110,2]  

In [12]:
alpha_bar=kron(alpha0,ones(n_x)')
alpha_bar=[alpha_bar alpha_bar];

In [13]:
# add constraints

@constraint(model,alpha0.>=0)
@constraint(model,Lambda.>=zeros(size(Lambda)))

for schedule_idx=1:n_schedules
    P_xi_w = S + S*Q[schedule_idx,:,:]*C_bar[schedule_idx,:,:]*S
    P_xi_v = S*Q[schedule_idx,:,:]
    P_xi_xi0 = ( I + S*Q[schedule_idx,:,:]*C_bar[schedule_idx,:,:] )*J

    R_T_mT = [ I ; -Matrix(1I, n_x*(T+1), n_x*(T+1)) ]

    # Add dual equality constraint
    LHS1 = Lambda[schedule_idx,:,:] * H_eta
    RHS1 = R_T_mT * [ P_xi_w P_xi_v P_xi_xi0 ]
    @constraint(model, LHS1 .== RHS1 )
    
    # Add dual inequality constraint
    LHS2 = Lambda[schedule_idx,:,:] * h_eta    
    RHS2 = alpha_bar[schedule_idx,:] - R_T_mT * S*r[schedule_idx,:]
    @constraint(model, LHS2.<=RHS2 )
    
    # low diagonal constraint
    Q_in = Q[schedule_idx,:,:]
    for blk_row_idx = 1:div(size(Q_in,1), n_x)-1
        LHS_bloc = Q_in[(blk_row_idx-1)*n_x.+(1:n_x),blk_row_idx*n_y+1:end]
        @constraint(model,LHS_bloc.==zeros(size(LHS_bloc)))
    end
end

In [14]:
@objective(model, Min, sum(alpha0))

alpha0[1,1] + alpha0[2,1] + alpha0[3,1] + alpha0[4,1] + alpha0[5,1] + alpha0[6,1] + alpha0[7,1] + alpha0[8,1] + alpha0[9,1] + alpha0[10,1] + alpha0[11,1] + alpha0[12,1] + alpha0[13,1] + alpha0[14,1] + alpha0[15,1] + alpha0[16,1] + alpha0[17,1] + alpha0[18,1] + alpha0[19,1] + alpha0[20,1] + alpha0[21,1] + alpha0[22,1] + alpha0[23,1] + alpha0[24,1] + alpha0[25,1] + alpha0[26,1] + alpha0[27,1] + alpha0[28,1] + alpha0[29,1] + alpha0[30,1] + alpha0[31,1] + alpha0[32,1] + alpha0[33,1] + alpha0[34,1] + alpha0[35,1] + alpha0[36,1] + alpha0[37,1] + alpha0[38,1] + alpha0[39,1] + alpha0[40,1] + alpha0[41,1] + alpha0[42,1] + alpha0[43,1] + alpha0[44,1] + alpha0[45,1] + alpha0[46,1] + alpha0[47,1] + alpha0[48,1] + alpha0[49,1] + alpha0[50,1] + alpha0[51,1] + alpha0[52,1] + alpha0[53,1] + alpha0[54,1] + alpha0[55,1] + alpha0[56,1] + alpha0[57,1] + alpha0[58,1] + alpha0[59,1] + alpha0[60,1] + alpha0[61,1] + alpha0[62,1] + alpha0[63,1] + alpha0[64,1] + alpha0[65,1] + alpha0[66,1] + alpha0[67,1] + alph

In [15]:
time_beginning=time_ns()

optimize!(model)

time_ending=time_ns()
exec_time=(time_ending-time_beginning)/1e9
print("\n\n=========== exec_time = $(exec_time) seconds ===========\n\n")

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 693480 rows, 495240 columns and 3219960 nonzeros
Model fingerprint: 0xffff4ebd
Coefficient statistics:
  Matrix range     [1e-13, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-07, 1e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 634950 rows and 373431 columns
Presolve time: 2.20s
Presolved: 58530 rows, 121809 columns, 607218 nonzeros

Ordering time: 0.15s

Barrier statistics:
 Free vars  : 9552
 AA' NZ     : 3.471e+06
 Factor NZ  : 4.993e+06 (roughly 110 MBytes of memory)
 Factor Ops : 5.475e+08 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          

In [16]:
print("termination_status = $(termination_status(model))\n")
print("primal_status = $(primal_status(model))\n")
print("dual_status = $(dual_status(model))\n")

val_alpha0s=value.(alpha0)
costs=sum(val_alpha0s,dims=2)

opt_idx=argmin(costs)
opt_cost=costs[opt_idx]
opt_schedule=all_schedules[opt_idx]

print("\nopt_schedule=$(opt_schedule), opt_cost=$(opt_cost)")


termination_status = OPTIMAL
primal_status = FEASIBLE_POINT
dual_status = FEASIBLE_POINT

opt_schedule=[2, 4, 7], opt_cost=42.562200000000004